# Importamos librerias

In [1]:
import pandas as pd
import numpy as np
import os
import snowflake.connector
import json
from snowflake.connector.pandas_tools import write_pandas
from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
pd.options.mode.chained_assignment = None

In [2]:
while True:
    try:

        f = open('credentials.json')
        data_pass = json.load(f)

        pass_ = input("INGRESAR PASSCODE:")

        ctx = snowflake.connector.connect(
            user = data_pass['snow']['USER'],
            password = data_pass['snow']['PASS'],
            account = data_pass['snow']['ACCOUNT'],
            passcode = pass_,
            database = 'SANDBOX_PLUS',
            schema = 'DWH'
        )

        cursor = ctx.cursor()

        print('Connected')

        break

    except:
        print('Incorrect Password - provide again')

Connected


# Subclases

In [3]:
########################################################################################################
########################################################################################################

ofertata = 'Marzo 3'

########################################################################################################
########################################################################################################
ofertata

'Marzo 3'

In [4]:
query = '''
SELECT
    DISTINCT
    SUB.SUB_NAME,
    SUB.SUBCLASE
FROM
    MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA
    INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
    INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = IM.SUBCLASE
WHERE
    LAA.ARTC_ARTC_COD IN ('244738000', '330191000', '252818000', '340343000', '250111000', '141428000', '140757000')
    AND SUB.SUB_NAME NOT ILIKE '%NO%ESPECIFICADO%'
'''

cursor.execute(query)
subclases = cursor.fetch_pandas_all()
subclases.head(2)

,SUB_NAME,SUBCLASE
0,CANOLA,1100010002
1,HARINA DE TRIGO 0000,1700030004


# Abandonadores

In [5]:
# antiguamente llamado Segmento 5
# Clientes que en el ultimo mes no vengan a tata y tengan Score 4 o 5 en la subclase

In [6]:
tuple(subclases['SUBCLASE'].astype(str))

('1100010002',
 '1700030004',
 '8800030003',
 '500040002',
 '2900020001',
 '100020002')

In [7]:
query = '''
WITH VINIERON AS
    (
    SELECT
        DISTINCT SOCI_SOCI_ID
    FROM
        MSTRDB.DWH.FT_FDLN_MOVIMIENTOS
    WHERE
        GEOG_LOCL_ID IN (SELECT GEOG_LOCL_ID FROM MSTRDB.DWH.LU_GEOG_LOCAL WHERE GEOG_UNNG_ID = 2)
        AND TIEM_DIA_ID >= DATEADD(MONTH, -1, CURRENT_DATE)
    )

SELECT
    SOCI_SOCI_ID,
    SUBCLASE,
    CORREO,
    CELULAR
FROM
    (
    SELECT
        LSS.SOCI_SOCI_ID,
        LSS.SUBCLASE,
        CONT.CORREO,
        CONT.CELULAR,
        ROW_NUMBER() OVER(PARTITION BY LSS.SOCI_SOCI_ID ORDER BY LSS.SOCI_SOCI_ID) AS R
    FROM
        BIZMETRIKS.DWH.LU_SCORING_SUBCLASE AS LSS
        INNER JOIN SANDBOX_PLUS.DWH.CONTACTABLES_2 AS CONT ON LSS.SOCI_SOCI_ID = CONT.SOCI_SOCI_ID
    WHERE
        LSS.SOCI_SOCI_ID NOT IN (SELECT * FROM VINIERON)
        AND LSS.SCORE >= 4
        AND LSS.SUBCLASE IN {subclases_snow}
        AND (CONT.CORREO IS NOT NULL OR CONT.CELULAR IS NOT NULL)
    )
WHERE
    R = 1
'''

cursor.execute(query.format(subclases_snow = tuple(subclases['SUBCLASE'].astype(str))))
segm_5 = cursor.fetch_pandas_all()

segm_5 = segm_5.merge(subclases, on = 'SUBCLASE', how = 'left')

print(segm_5.shape[0])
segm_5.head(2)

145577


,SOCI_SOCI_ID,SUBCLASE,CORREO,CELULAR,SUB_NAME
0,1749445,2900020001,None,99860906,HAMBURGUESAS VACUNA EN CAJA
1,2113502,1100010002,None,93608135,CANOLA


# Activos

In [19]:
# antiguamente llamado Segmento 3
# Para cada categoria (subclase) son los Clientes activos que no compran la subclase y tienen score 5

In [8]:
query = '''
WITH ACTIVOS AS
    (
    SELECT
        DISTINCT SOCI_SOCI_ID
    FROM
        MSTRDB.DWH.FT_FDLN_MOVIMIENTOS
    WHERE
        GEOG_LOCL_ID IN (SELECT GEOG_LOCL_ID FROM MSTRDB.DWH.LU_GEOG_LOCAL WHERE GEOG_UNNG_ID = 2)
        AND TIEM_DIA_ID >= DATEADD(MONTH, -1, CURRENT_DATE)
    ),

COMPRAS_SUBCLASES AS
    (
    SELECT
        DISTINCT
        FFM.SOCI_SOCI_ID || ' - ' || IM.SUBCLASE AS KEY
    FROM
        MSTRDB.DWH.FT_FDLN_MOVIMIENTOS AS FFM
        INNER JOIN MSTRDB.DWH.FT_VENTAS AS FV ON FV.TICKET = FFM.TICKET
        INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON FV.ARTC_ARTC_ID = LAA.ARTC_ARTC_ID
        INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
    WHERE
        FFM.GEOG_LOCL_ID IN (SELECT GEOG_LOCL_ID FROM MSTRDB.DWH.LU_GEOG_LOCAL WHERE GEOG_UNNG_ID = 2)
        AND FFM.TIEM_DIA_ID >= DATEADD(MONTH, -3, CURRENT_DATE)
    )


SELECT
    SOCI_SOCI_ID,
    SUBCLASE,
    CORREO,
    CELULAR
FROM
    (
    SELECT
        LSS.SOCI_SOCI_ID,
        LSS.SUBCLASE,
        CONT.CORREO,
        CONT.CELULAR,
        ROW_NUMBER() OVER(PARTITION BY LSS.SOCI_SOCI_ID ORDER BY LSS.SOCI_SOCI_ID) AS R
    FROM
        BIZMETRIKS.DWH.LU_SCORING_SUBCLASE AS LSS
        INNER JOIN SANDBOX_PLUS.DWH.CONTACTABLES_2 AS CONT ON LSS.SOCI_SOCI_ID = CONT.SOCI_SOCI_ID
    WHERE
        LSS.SOCI_SOCI_ID IN (SELECT * FROM ACTIVOS)
        AND LSS.SCORE = 5
        AND LSS.SUBCLASE IN {subclases_snow}
        AND LSS.SOCI_SOCI_ID || ' - ' || LSS.SUBCLASE NOT IN (SELECT * FROM COMPRAS_SUBCLASES)
        AND (CONT.CORREO IS NOT NULL OR CONT.CELULAR IS NOT NULL)
    )
WHERE
    R = 1
'''

cursor.execute(query.format(subclases_snow = tuple(subclases['SUBCLASE'].astype(str))))
segm_3 = cursor.fetch_pandas_all()

segm_3 = segm_3.merge(subclases, on = 'SUBCLASE', how = 'left')

print(segm_3.shape[0])
segm_3.head(2)

155184


,SOCI_SOCI_ID,SUBCLASE,CORREO,CELULAR,SUB_NAME
0,1505628,2900020001,None,97326058,HAMBURGUESAS VACUNA EN CAJA
1,39512,1100010002,None,99044133,CANOLA


# Excels

In [9]:
for i in segm_3['SUB_NAME'].unique():
    print(i)
    df_aux = segm_3[segm_3['SUB_NAME'] == i]
    df_aux.to_excel(f'Ofertata {ofertata} Activos - {i}.xlsx', index = False)

HAMBURGUESAS VACUNA EN CAJA
CANOLA
CUADRADA
CERVEZAS IMPORTADAS
JABONES DETERGENTE LIQUIDOS PARA ROPA
HARINA DE TRIGO 0000


In [10]:
for i in segm_5['SUB_NAME'].unique():
    print(i)
    df_aux = segm_5[segm_5['SUB_NAME'] == i]
    df_aux.to_excel(f'Ofertata {ofertata} Abandonadores - {i}.xlsx', index = False)

HAMBURGUESAS VACUNA EN CAJA
CANOLA
JABONES DETERGENTE LIQUIDOS PARA ROPA
CUADRADA
CERVEZAS IMPORTADAS
HARINA DE TRIGO 0000


In [23]:
sys.exit()

SystemExit: 

C:\Users\leonardo.mangold\Desktop\Tareas\Analisis\!Python\env\lib\site-packages\IPython\core\interactiveshell.py:3406: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# SNOW

In [13]:
ofertata.upper()

'MARZO 3'

In [14]:
segm_5['PERFIL_CLIENTE'] = 'ABANDONADOR'
segm_3['PERFIL_CLIENTE'] = 'ACTIVO'
ambos = pd.concat([segm_5, segm_3])
ambos['OFERTATA'] = ofertata.upper()
ambos['FECHA'] = (datetime.today() + timedelta(days = 1)).strftime('%Y-%m-%d')
ambos.head(2)

,SOCI_SOCI_ID,SUBCLASE,CORREO,CELULAR,SUB_NAME,PERFIL_CLIENTE,OFERTATA,FECHA
0,1749445,2900020001,None,99860906,HAMBURGUESAS VACUNA EN CAJA,ABANDONADOR,MARZO 3,2024-03-14
1,2113502,1100010002,None,93608135,CANOLA,ABANDONADOR,MARZO 3,2024-03-14


In [15]:
success, nchunks, nrows, _ = write_pandas(ctx,
    ambos.drop(['SUB_NAME'], axis = 1), table_name='SEGMENTOS_OFERTATA', database= 'SANDBOX_PLUS', schema = 'DWH')
print('listo')

listo


# Litoral

## 6 meses

In [16]:
query = '''
WITH LOCAL_DOMINANTE AS
    (
    SELECT
        DISTINCT
        SOCI_SOCI_ID,
        DOMINANTE,
        GEOG_LOCL_DESC
    FROM
        (
        SELECT
            FFM.SOCI_SOCI_ID,
            LGL.GEOG_LOCL_COD AS DOMINANTE,
            LGL.GEOG_LOCL_DESC,
            SUM(FV.VNTA_IMPORTE_SIN_IVA) AS VENTAS,
            ROW_NUMBER() OVER (PARTITION BY SOCI_SOCI_ID ORDER BY VENTAS DESC) AS RNK
        FROM
            MSTRDB.DWH.FT_VENTAS AS FV
            INNER JOIN MSTRDB.DWH.FT_FDLN_MOVIMIENTOS AS FFM ON FV.TICKET = FFM.TICKET
            INNER JOIN MSTRDB.DWH.LU_GEOG_LOCAL AS LGL ON FV.GEOG_LOCL_ID = LGL.GEOG_LOCL_ID
            INNER JOIN MSTRDB.DWH.LU_GEOG_DEPTO AS LGD ON LGD.GEOG_DPTO_ID = LGL.GEOG_DPTO_ID
            INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON LAA.ARTC_ARTC_ID = FV.ARTC_ARTC_ID
        WHERE
            FV.TIEM_DIA_ID BETWEEN  DATEADD(MONTH, -24, CURRENT_DATE()) AND CURRENT_DATE()
            AND LGL.GEOG_UNNG_ID = 2
            AND LAA.ARTC_SECC_ID <> 42
            AND FFM.FDLN_MOVT_TIPO LIKE '%RP%'
        GROUP BY
            ALL
        )
    WHERE
        RNK = 1
        AND DOMINANTE IN (111, 152, 156, 158, 159, 160, 161, 162, 316, 125, 127, 317)
    ),

VINIERON AS
    (
    SELECT
        DISTINCT SOCI_SOCI_ID
    FROM
        MSTRDB.DWH.FT_FDLN_MOVIMIENTOS
    WHERE
        GEOG_LOCL_ID IN (SELECT GEOG_LOCL_ID FROM MSTRDB.DWH.LU_GEOG_LOCAL WHERE GEOG_UNNG_ID = 2)
        AND TIEM_DIA_ID >= DATEADD(MONTH, -6, CURRENT_DATE)
    )

    SELECT
        SOCI_SOCI_ID,
        CORREO,
        CELULAR
    FROM
        SANDBOX_PLUS.DWH.CONTACTABLES_2
    WHERE
        SOCI_SOCI_ID NOT IN (SELECT * FROM VINIERON)
        AND SOCI_SOCI_ID IN (SELECT SOCI_SOCI_ID FROM LOCAL_DOMINANTE)
        AND (CORREO IS NOT NULL OR CELULAR IS NOT NULL)

'''

cursor.execute(query.format(subclases_snow = tuple(subclases['SUBCLASE'].astype(str))))
litoral_abandonador_6_meses = cursor.fetch_pandas_all()

litoral_abandonador_6_meses = litoral_abandonador_6_meses[~litoral_abandonador_6_meses['SOCI_SOCI_ID'].isin(ambos['SOCI_SOCI_ID'])]

In [17]:
litoral_abandonador_6_meses_2 = pd.DataFrame()

for subclase, subclase_name in zip(subclases['SUBCLASE'], subclases['SUB_NAME']):
    print(subclase, subclase_name)
    df_aux = litoral_abandonador_6_meses.copy(deep = True)
    df_aux['SUBCLASE'] = subclase
    df_aux['SUB_NAME'] = subclase_name
    litoral_abandonador_6_meses_2 = pd.concat([litoral_abandonador_6_meses_2, df_aux])

litoral_abandonador_6_meses_2.shape[0] == litoral_abandonador_6_meses.shape[0] * subclases.shape[0]

1100010002 CANOLA
1700030004 HARINA DE TRIGO 0000
8800030003 JABONES DETERGENTE LIQUIDOS PARA ROPA
500040002 CUADRADA
2900020001 HAMBURGUESAS VACUNA EN CAJA
100020002 CERVEZAS IMPORTADAS


True

In [18]:
litoral_abandonador_6_meses_2.head(2)

,SOCI_SOCI_ID,CORREO,CELULAR,SUBCLASE,SUB_NAME
0,996982,None,99649403,1100010002,CANOLA
1,1875933,None,92927805,1100010002,CANOLA


In [19]:
for i in litoral_abandonador_6_meses_2['SUB_NAME'].unique():
    print(i)
    df_aux = litoral_abandonador_6_meses_2[litoral_abandonador_6_meses_2['SUB_NAME'] == i].drop(['SOCI_SOCI_ID', 'SUBCLASE', 'SUB_NAME'], axis = 1)
    df_aux.to_excel(f'Ofertata {ofertata} Abandonador 6 meses Litoral - {i}.xlsx', index = False)
print('listo')

CANOLA
HARINA DE TRIGO 0000
JABONES DETERGENTE LIQUIDOS PARA ROPA
CUADRADA
HAMBURGUESAS VACUNA EN CAJA
CERVEZAS IMPORTADAS
listo


In [20]:
litoral_abandonador_6_meses_2['PERFIL_CLIENTE'] = 'ABANDONADOR'
litoral_abandonador_6_meses_2['OFERTATA'] = ofertata.upper()
litoral_abandonador_6_meses_2['FECHA'] = (datetime.today() + timedelta(days = 1)).strftime('%Y-%m-%d')

success, nchunks, nrows, _ = write_pandas(ctx,
    litoral_abandonador_6_meses_2.drop(['SUB_NAME'], axis = 1), table_name='SEGMENTOS_OFERTATA', database= 'SANDBOX_PLUS', schema = 'DWH')
print('listo')

listo


# Fin